# retirement calculation

given: life expectancy, current age, retirement age, account value at retirement (how much money you have when you retire, at the time of your retirement), expenditure after retirement (how much money you need to spend, at today's value before inflation)

simulate: inflation, investment return. three ways of number generation (1) assumed constant (2) normal distribution randomiser based on historical returns within a pre-defined time range (2) bootstrapping from historical returns within a pre-defined time range  

calculate: success rate (success = account never dip below safety threshold throughout lifetime), life-end account value distribution

assume: no major change of lifestyle during retirement that requires deviation from usual expenditure pattern. portfolio stays at the same composition (monthly rebalanced)

In [1]:
import sys
project_root = r"/Users/cococao/Desktop/retirement_calculator"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import os
from pathlib import Path
from dotenv import load_dotenv
# locate .env (prefer project root, fall back to current working directory)
dotenv_path = Path(project_root) / ".env"
if not dotenv_path.exists():
    dotenv_path = Path(".") / ".env"

if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path, override=False)

In [5]:
retirement_capital = 1000000
retirement_age = 40

return_method = 'normal'
proportion_stock = 0.6
start_year = 1962 # earliest data for historical investment return is from 1928
end_year = 2024
n_simulation = 5000
inflation_country = "mixed" # data available for US, UK, singapore, spain. mixed averages over all available countries
threshold_failure=10000 # the account balance below which the simulation is considered a failure if any month falls below it

In [6]:
# for after retirement 
from utilities.simulators.drawdown.paralle_worker_simulator import ParallelWorkerSimulator

pws = ParallelWorkerSimulator()
pws.simulate(retirement_capital=retirement_capital,retirement_age=retirement_age,n_simulation=n_simulation, start_year=start_year, end_year=end_year, return_method=return_method, proportion_stock=proportion_stock, threshold_failure=threshold_failure)

{'final_account_analysis': {'mean': np.float64(3892199.459532905),
  'median': np.float64(3669700.2613107585),
  'percentile_10': np.float64(1650217.3243878677),
  'percentile_90': np.float64(6454248.13584012)},
 'failure_rate': np.float64(0.0048)}

# whole-path calculation
including the accumulative phase (saving before retirement). to calculate how early I can retire at what income & expenditure level.

given: life expectancy, current age, retirement age, current account value (how much money you have now), current expenditure (at today's value, before inflation), post-retirement expenditure (at today's value, before inflation), average monthly income during accumulation phase, average annual income increment during accumulation phase

simulate: inflation, investment return. three ways of number generation (1) assumed constant (2) normal distribution randomiser based on historical returns within a pre-defined time range (2) bootstrapping from historical returns within a pre-defined time range  

calculate: success rate (success = account never dip below safety threshold throughout lifetime), life-end account value distribution

assume: no lifestyle inflation during accumulation phase. no major unexpected expenditure during retirement. portfolio stays at the same composition (monthly rebalanced). constant pre-defined annual income increment rate during accumulation phase.

In [4]:
# overall, with accumulation

monthly_net_income = 6000
retirement_age = 40

return_method = 'normal'
proportion_stock = 1
start_year = 1962 # earliest data for historical investment return is from 1928
end_year = 2024
n_simulation = 5000
inflation_country = "mixed" # data available for US, UK, singapore, spain. mixed averages over all available countries
threshold_failure=10000


In [5]:
from utilities.simulators.whole_path.paralle_worker_simulator import ParallelWorkerSimulator
pws = ParallelWorkerSimulator()
pws.simulate(monthly_net_income=monthly_net_income, retirement_age=retirement_age,n_simulation=n_simulation, start_year=start_year, end_year=end_year, return_method=return_method, proportion_stock=proportion_stock, threshold_failure=threshold_failure)

{'final_account_analysis': {'mean': np.float64(7522011.416807744),
  'median': np.float64(6563950.731937225),
  'percentile_10': np.float64(1956808.750139952),
  'percentile_90': np.float64(14249877.744433247)},
 'failure_rate': np.float64(0.015)}

# Asset allocation tester

We want to (1) retire early (2) reduce risk of account failure in lifetime to reasonably low levels. Given our customised set of assumptions about , here I test out a few strategies of asset allocation to see which one suits our specific use case better.

strategies tested:
- constant: stock composition stays constant at a pre-defined rate, monthly rebalanced
- glidepath: 100% stocks until retirement. after retirement, gradually reduce stocks composition until it reaches zero at the end of life expectancy
- moderator: start with a pre-defined stock composition. if the present year garners higher return than historical average for that composition, do more bonds next year. if the present year garners lower return, do more stocks. if similar to historical average, remains. annually rebalanced
- polariser: like moderator but opposite direction of change in response to return deviation
- moderator_historical: like moderator, but instead of comparing last-year return against historical median, compare overall account against if all past returns have been historical median
- polariser_historical: like moderator_historical but opposite direction of change in response to account value deviation


In [2]:
# overall, with accumulation
monthly_net_income = 7000
retirement_age = 35

return_method = 'normal'
start_year = 1962 # earliest data for historical investment return is from 1928
end_year = 2024
n_simulation = 5000
inflation_country = "mixed" # data available for US, UK, singapore, spain. mixed averages over all available countries
threshold_failure=10000


In [10]:
# test moderator 
from utilities.simulators.whole_path.paralle_worker_simulator import ParallelWorkerSimulator
pws = ParallelWorkerSimulator()

res = pws.simulate(monthly_net_income=monthly_net_income, retirement_age=retirement_age,n_simulation=n_simulation, start_year=start_year, end_year=end_year, threshold_failure=threshold_failure, f_portfolio_composition = True, portfolio_method = 'moderator', start_proportion = 1.0, deviation_threshold = 0.6, sensitivity = 0.2)
res

{'final_account_analysis': {'mean': np.float64(2483391.7546208366),
  'median': np.float64(1912847.7555054317),
  'percentile_10': np.float64(-1019094.8665576822),
  'percentile_90': np.float64(6715017.640973894)},
 'failure_rate': np.float64(0.22564)}

In [7]:
from utilities.simulators.whole_path.paralle_worker_simulator import ParallelWorkerSimulator
pws = ParallelWorkerSimulator()

dic_testing_strategies_results = {}

for k in range(11):
    stock_proportion = round(k/10,1)
    testing_strategy = ('constant', 'stock_proportion = ' + str(stock_proportion))
    res = pws.simulate(monthly_net_income=monthly_net_income, retirement_age=retirement_age,n_simulation=n_simulation, start_year=start_year, end_year=end_year, return_method=return_method, proportion_stock=stock_proportion, threshold_failure=threshold_failure)
    dic_testing_strategies_results[testing_strategy] = res

for glidepath_type in ['linear', 'concave_down', 'concave_up']:
    dic_testing_strategies_results[('glidepath', glidepath_type)] = pws.simulate(monthly_net_income=monthly_net_income, retirement_age=retirement_age,n_simulation=n_simulation, start_year=start_year, end_year=end_year, return_method=return_method, threshold_failure=threshold_failure, f_portfolio_composition = True, portfolio_method = 'glidepath', glidepath_type = glidepath_type)

In [13]:
import pandas as pd

l_methods = list(dic_testing_strategies_results.keys())
df_portfolio_composition_comparison = pd.DataFrame({
    'method': l_methods,
    'failure_rate': [dic_testing_strategies_results[method]['failure_rate'] for method in l_methods],
    'portfolio_median': [dic_testing_strategies_results[method]['final_account_analysis']['median'] for method in l_methods],
    'portfolio_10th_percentile':  [dic_testing_strategies_results[method]['final_account_analysis']['percentile_10'] for method in l_methods],
    'portfolio_90th_percentile': [dic_testing_strategies_results[method]['final_account_analysis']['percentile_90'] for method in l_methods]
})

df_portfolio_composition_comparison.sort_values(by='failure_rate')

,method,failure_rate,portfolio_median,portfolio_10th_percentile,portfolio_90th_percentile
5,"(constant, stock_proportion = 0.5)",0.2120,1.125429e+06,-5.446247e+05,3.453858e+06
4,"(constant, stock_proportion = 0.4)",0.2182,8.950382e+05,-4.983637e+05,2.682194e+06
7,"(constant, stock_proportion = 0.7)",0.2188,1.507640e+06,-7.646252e+05,4.887536e+06
8,"(constant, stock_proportion = 0.8)",0.2208,1.586956e+06,-9.091274e+05,5.692797e+06
6,"(constant, stock_proportion = 0.6)",0.2252,1.231981e+06,-7.046478e+05,4.000447e+06
9,"(constant, stock_proportion = 0.9)",0.2328,1.842763e+06,-1.000714e+06,6.543239e+06
10,"(constant, stock_proportion = 1.0)",0.2392,1.949196e+06,-1.231937e+06,7.594694e+06
13,"(glidepath, concave_up)",0.2420,1.808414e+06,-1.157134e+06,6.305490e+06
11,"(glidepath, linear)",0.2480,1.490702e+06,-1.057093e+06,5.254091e+06
12,"(glidepath, concave_down)",0.2488,1.311572e+06,-9.721029e+05,4.391437e+06


# When can I retire?

given info about income and expenditures, considering all available strategies, calculate at which age I can possibly retire

In [2]:
# overall, with accumulation
monthly_net_income = 7000
max_tolerated_failure_rate = 0.05


return_method = 'normal'
start_year = 1962 # earliest data for historical investment return is from 1928
end_year = 2024
n_simulation = 1000
inflation_country = "mixed" # data available for US, UK, singapore, spain. mixed averages over all available countries
threshold_failure=10000

l_available_strategies = []
for k in range(6):
    stock_proportion = round(k/5,1)
    constant_strategy = ('constant', stock_proportion)
    l_available_strategies.append(constant_strategy)
for glidepath_type in ['linear', 'concave_down', 'concave_up']:
    l_available_strategies.append(('glidepath', glidepath_type))
for strategy_type in ['moderator', 'polariser']:
    for starting_stock_proportion in [1.0, 0.6, 0.0]:
        l_available_strategies.append((strategy_type, starting_stock_proportion))


In [3]:
from utilities.calculators.retirement_age_calculator import RetirementAgeCalculator

c_retirementAge = RetirementAgeCalculator()

c_retirementAge.calculate(monthly_net_income = monthly_net_income, l_available_strategies = l_available_strategies, max_tolerated_failure_rate = max_tolerated_failure_rate,
        n_simulation=n_simulation,
        return_method=return_method,
        threshold_failure=threshold_failure,start_year = start_year,end_year = end_year,inflation_country=inflation_country)

Congrats, you can retire within your lifetime. Calculating earliest retirement age now...
Age 31 fails. At this age, your lowest failure rate is 0.984
Age 32 fails. At this age, your lowest failure rate is 0.877
Age 33 fails. At this age, your lowest failure rate is 0.664
Age 34 fails. At this age, your lowest failure rate is 0.425
Age 35 fails. At this age, your lowest failure rate is 0.202
Success! At age 36, you will be able to retire! Your lowest failure rate is 0.02
Printing results for all eligible portfolio strategies:
            method  failure_rate  portfolio_median  portfolio_10th_percentile  \
1  (constant, 0.2)         0.020      1.635233e+06              549209.454674   
0  (constant, 0.0)         0.036      1.230746e+06              326738.663220   
3  (constant, 0.6)         0.037      2.613564e+06              545269.890862   
2  (constant, 0.4)         0.045      2.121660e+06              488847.455505   

   portfolio_90th_percentile  
1               3.042597e+06  


,method,failure_rate,portfolio_median,portfolio_10th_percentile,portfolio_90th_percentile
1,"(constant, 0.2)",0.020,1.635233e+06,549209.454674,3.042597e+06
0,"(constant, 0.0)",0.036,1.230746e+06,326738.663220,2.387086e+06
3,"(constant, 0.6)",0.037,2.613564e+06,545269.890862,5.691211e+06
2,"(constant, 0.4)",0.045,2.121660e+06,488847.455505,4.154112e+06


In [16]:
from utilities.portfolio_compositors.moderator import Moderator
from utilities.retirement_account_drawdown_dynamic_portfolio import RetirementAccountCalculator_DynamicPortfolio

strategy_m = Moderator()
c_retirement = RetirementAccountCalculator_DynamicPortfolio()

c_retirement.monthly_account(strategy_m, f_dynamic_composition_annual_verbose = True, deviation_threshold = 0.6,return_method='normal')

dynamic composer starts with stock ratio = 0.6
dynamic composer changes portfolio stock ratio to 0.4
dynamic composer changes portfolio stock ratio to 0.6
dynamic composer changes portfolio stock ratio to 0.8
dynamic composer changes portfolio stock ratio to 1.0


[800000,
 np.float64(782229.9567232252),
 np.float64(793602.970124809),
 np.float64(800144.1464234175),
 np.float64(811867.3219774062),
 np.float64(825968.7421124544),
 np.float64(826432.9989486662),
 np.float64(834153.383956149),
 np.float64(849849.1300559399),
 np.float64(857033.1411603243),
 np.float64(843601.6851258838),
 np.float64(840241.7096545094),
 np.float64(849596.3037882226),
 np.float64(859458.2922494251),
 np.float64(861987.8821490865),
 np.float64(884844.5482754641),
 np.float64(884109.6493186593),
 np.float64(891989.5178251965),
 np.float64(906039.5730340623),
 np.float64(921111.244435932),
 np.float64(921096.5680241634),
 np.float64(925944.4000819626),
 np.float64(934077.9740488875),
 np.float64(927705.6527880467),
 np.float64(933244.340652503),
 np.float64(943710.8648674634),
 np.float64(945123.7655228814),
 np.float64(953190.5948318519),
 np.float64(948275.1943100042),
 np.float64(950918.2070167995),
 np.float64(951379.3603393892),
 np.float64(954396.1545148742),
 np